In [ ]:
!pip install nibabel scikit-learn scipy torchvision torchio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
import torch
from torch.utils.data import Dataset
import torchio as tio

class MRIDataset(Dataset):
    def __init__(self, root_dir, target_shape=(32, 112, 112), ad_repeat=2, augment=True):
        self.file_paths = []
        self.labels = []
        self.target_shape = target_shape
        self.augment = augment

        self.transform = tio.Compose([
            tio.RandomFlip(axes=('LR', 'AP', 'IS'), p=0.5),
            tio.RandomAffine(scales=(0.9, 1.1), degrees=10, translation=5, p=0.7),
            tio.RandomNoise(mean=0.0, std=0.1, p=0.4),
            tio.RandomBiasField(p=0.3),
        ])

        class_folders = ['CN', 'MCI', 'AD']
        for label, class_name in enumerate(class_folders):
            class_path = os.path.join(root_dir, class_name)
            if not os.path.exists(class_path): continue

            for fname in os.listdir(class_path):
                file_path = os.path.join(class_path, fname)
                if not os.path.isfile(file_path): continue
                if not (fname.endswith('.nii') or fname.endswith('.nii.gz')): continue  # sadece NIfTI dosyaları

                repeat = ad_repeat if class_name == 'AD' else 1
                for _ in range(repeat):
                    self.file_paths.append(file_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        label = self.labels[idx]

        img = nib.load(path).get_fdata()
        img = np.nan_to_num(img)

        std = np.std(img)
        if std == 0:
            img = img - np.mean(img)
        else:
            img = (img - np.mean(img)) / std

        zoom_factors = [t / s for t, s in zip(self.target_shape, img.shape)]
        img = zoom(img, zoom_factors, order=1)
        img = np.expand_dims(img, axis=0)  # [1, D, H, W]

        if self.augment:
            subject = tio.Subject(mri=tio.ScalarImage(tensor=torch.tensor(img)))
            transformed = self.transform(subject)
            img = transformed.mri.data

        return torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
import torch
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import random

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import random_split, DataLoader

# Veri yolu
data_dir = '/content/drive/MyDrive/Processed'

# Dataset'i yükle (AD çoğaltması dahil)
dataset = MRIDataset(data_dir, ad_repeat=2)

# Dataset kontrolü
print(f"Toplam örnek sayısı: {len(dataset)}")

if len(dataset) > 0:
    print("✅ Dosyalar bulundu. İlk 5 dosya yolu:")
    for i in range(min(5, len(dataset))):
        print(dataset.file_paths[i])
else:
    raise ValueError("❌ Hiç dosya bulunamadı. Lütfen klasör yapısını ve yolunu kontrol et.")

# Dataset bölme (80% train, 10% val, 10% test)
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size  # kalan

train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

# Validation ve Test için augmentasyonu kapatalım
val_ds.dataset.augment = False
test_ds.dataset.augment = False

# DataLoader'lar
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)

print(f"\n🔍 Dataset Bölümü:")
print(f"Train: {len(train_ds)} | Val: {len(val_ds)} | Test: {len(test_ds)}")



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

# 3D DenseLayer: DenseNet'in yapı taşı
class _DenseLayer3D(nn.Module):
    def __init__(self, in_channels, growth_rate, bn_size=4, drop_rate=0):
        super().__init__()
        self.norm1 = nn.BatchNorm3d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv3d(in_channels, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)

        self.norm2 = nn.BatchNorm3d(bn_size * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(bn_size * growth_rate, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)

        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = self.conv1(self.relu1(self.norm1(x)))
        new_features = self.conv2(self.relu2(self.norm2(new_features)))
        if self.drop_rate > 0:
            new_features = F.dropout3d(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


# 3D DenseBlock
class _DenseBlock3D(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate, bn_size=4, drop_rate=0):
        super().__init__()
        layers = []
        for i in range(num_layers):
            layers.append(_DenseLayer3D(in_channels + i * growth_rate, growth_rate, bn_size, drop_rate))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)


# 3D Transition layer
class _Transition3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.norm = nn.BatchNorm3d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.pool = nn.AvgPool3d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(self.relu(self.norm(x)))
        x = self.pool(x)
        return x


# 3D DenseNet modeli
class DenseNet3D(nn.Module):
    def __init__(self, num_classes=3, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv3d(1, num_init_features, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm3d(num_init_features),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        )

        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock3D(num_layers, num_features, growth_rate, bn_size, drop_rate)
            self.features.add_module(f'denseblock{i+1}', block)
            num_features += num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition3D(num_features, num_features // 2)
                self.features.add_module(f'transition{i+1}', trans)
                num_features = num_features // 2

        self.features.add_module('norm_final', nn.BatchNorm3d(num_features))

        self.classifier = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool3d((1, 1, 1)),
            nn.Flatten(),
            nn.Linear(num_features, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes)
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm3d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                init.constant_(m.bias, 0)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet3D(num_classes=3).to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def evaluate_model(loader, name):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    acc = correct / total * 100
    print(f"{name} | Accuracy: {acc:.2f}% | Loss: {avg_loss:.4f}")

    # Ek metrikler
    print("\n📊 Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=["CN", "MCI", "AD"], digits=4))

    print("🧩 Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    return avg_loss, acc

# Eğitim parametreleri
num_epochs = 5
patience = 3
best_val_loss = float('inf')
epochs_without_improvement = 0

print(f"\nTrain: {len(train_ds)} | Val: {len(val_ds)} | Test: {len(test_ds)}")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        if (batch_idx + 1) % 2 == 0 or (batch_idx + 1) == len(train_loader):
            acc = correct / total * 100
            print(f"  Batch {batch_idx+1} | Loss: {loss.item():.4f} | Accuracy: {acc:.2f}%")

    avg_train_loss = total_loss / len(train_loader)
    train_acc = correct / total * 100
    print(f"Epoch {epoch+1} Sonu | Train Loss: {avg_train_loss:.4f} | Train Accuracy: {train_acc:.2f}%")

    # Doğrulama
    val_loss, val_acc = evaluate_model(val_loader, f"Validation (Epoch {epoch+1})")

    # Erken durdurma kontrolü
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        print(f"  ⚠️ Validation loss iyileşmedi. ({epochs_without_improvement}/{patience})")
        if epochs_without_improvement >= patience:
            print(f"  🛑 Validation loss {patience} epoch boyunca iyileşmedi. Eğitim durduruluyor.")
            break

# Final test
print("\n=== Final Test Evaluation ===")
test_loss, test_acc = evaluate_model(test_loader, "Test Set")


Train: 2372 | Val: 263 | Test: 263

Epoch 1/5
  Batch 2 | Loss: 1.1739 | Accuracy: 28.12%
  Batch 4 | Loss: 1.1930 | Accuracy: 29.69%
  Batch 6 | Loss: 1.0831 | Accuracy: 32.29%
  Batch 8 | Loss: 1.0676 | Accuracy: 34.38%
  Batch 10 | Loss: 1.0641 | Accuracy: 36.88%
  Batch 12 | Loss: 1.1358 | Accuracy: 37.76%
  Batch 14 | Loss: 1.0822 | Accuracy: 38.84%
  Batch 16 | Loss: 0.9987 | Accuracy: 39.45%
  Batch 18 | Loss: 1.0695 | Accuracy: 39.41%
  Batch 20 | Loss: 1.1648 | Accuracy: 39.38%
  Batch 22 | Loss: 1.1279 | Accuracy: 38.78%
  Batch 24 | Loss: 1.0505 | Accuracy: 39.71%
  Batch 26 | Loss: 1.1140 | Accuracy: 39.30%
  Batch 28 | Loss: 1.0624 | Accuracy: 39.29%
  Batch 30 | Loss: 1.0536 | Accuracy: 39.90%
  Batch 32 | Loss: 1.1119 | Accuracy: 39.94%
  Batch 34 | Loss: 1.1017 | Accuracy: 39.43%
  Batch 36 | Loss: 1.0162 | Accuracy: 40.28%
  Batch 38 | Loss: 1.1037 | Accuracy: 40.54%
  Batch 40 | Loss: 0.9961 | Accuracy: 41.33%
  Batch 42 | Loss: 1.0662 | Accuracy: 41.59%
  Batch 44 |